# TensorBoard Integration with LatticeMC

This notebook demonstrates how to use TensorBoard to visualize simulation progress with immediate logging.

In [1]:
import numpy as np
from decimal import Decimal

In [2]:
from latticemc.definitions import Lattice, LatticeState, OrderParametersHistory, DefiningParameters
from latticemc.lattice_tools import initialize_partially_ordered
from latticemc.random_quaternion import random_quaternion
from latticemc.updaters import AcceptanceRateWiggleRateAdjustor, RandomWiggleRateAdjustor
from latticemc.parallel import SimulationRunner, ProgressBarMode

## Set up simulation with TensorBoard logging enabled

We'll create a simulation with multiple temperatures and enable TensorBoard logging.
With the new immediate logging approach, data is logged as soon as it's received from simulation processes.

In [3]:
# Setup simulation with multiple temperatures
temperatures = np.arange(0.3, 1.7, 0.1)  # Fewer temps for faster demo
states = [LatticeState(parameters=DefiningParameters(temperature=round(Decimal(t), 2), lam=Decimal(0.3), tau=1),
                       lattice=Lattice(8, 8, 8))  # Smaller lattice for faster demo
          for t in temperatures]
for state in states:
    initialize_partially_ordered(state.lattice, x=random_quaternion(1.0))

order_parameters_history = {state.parameters: OrderParametersHistory() for state in states}

In [4]:
# Setup updaters
per_state_updaters = [
    AcceptanceRateWiggleRateAdjustor(how_often=100, since_when=100),
    RandomWiggleRateAdjustor(scale=0.001, how_often=100, since_when=100),
    RandomWiggleRateAdjustor(scale=1.0, reset_value=1.0, how_often=500, since_when=500)
]

# Create simulation runner with TensorBoard logging and progress bars enabled
# TensorBoard logs will automatically be saved to working_folder/tensorboard
runner = SimulationRunner(states,
                         order_parameters_history,
                         cycles=1000,
                         report_order_parameters_every=50,
                         report_state_every=50,
                         per_state_updaters=per_state_updaters,
                         parallel_tempering_interval=50,
                         save_interval=100,
                         plot_recent_points=1000,
                         working_folder='./tensorboard_demo',  # TensorBoard logs go to ./tensorboard_demo/tensorboard
                         progress_bar_mode=ProgressBarMode.NOTEBOOK)

## Start the simulation

Run the simulation and TensorBoard logging will happen automatically as data is received:
- Order parameters are logged immediately when received
- Fluctuations are logged immediately when received  
- Lattice states (including energy and acceptance rates) are logged when state updates are received

## Progress Bars

The simulation now includes progress bars to track the simulation progress:
- **Main progress bar**: Shows overall simulation progress across all temperatures
- **Per-simulation progress bars**: Individual progress for each temperature simulation

These progress bars work seamlessly with TensorBoard logging and provide real-time feedback on simulation status.

In [ ]:
# Start the simulation
runner.start()

Overall Progress:   0%|          | 0/14000 [00:00<?, ?steps/s]

Sim 0 (T=0.300):   0%|          | 0/1000 [00:00<?, ?steps/s]

Sim 1 (T=0.400):   0%|          | 0/1000 [00:00<?, ?steps/s]

Sim 2 (T=0.500):   0%|          | 0/1000 [00:00<?, ?steps/s]

Sim 3 (T=0.600):   0%|          | 0/1000 [00:00<?, ?steps/s]

...

Sim 10 (T=1.300):   0%|          | 0/1000 [00:00<?, ?steps/s]

Sim 11 (T=1.400):   0%|          | 0/1000 [00:00<?, ?steps/s]

Sim 12 (T=1.500):   0%|          | 0/1000 [00:00<?, ?steps/s]

Sim 13 (T=1.600):   0%|          | 0/1000 [00:00<?, ?steps/s]

All simulation processes finished gracefully


## Launch TensorBoard

While the simulation is running, you can launch TensorBoard in a new terminal to visualize the progress:

```bash
tensorboard --logdir=./tensorboard_demo/tensorboard
```

This will start a local server, typically at http://localhost:6006, where you can view the visualizations.

### What you'll see in TensorBoard:

1. **Order Parameters**: Real-time plots of energy, q0, q2, w, p, d322 for each temperature
2. **Fluctuations**: Real-time fluctuation measurements for each order parameter
3. **Acceptance Rates**: Live monitoring of orientation and parity move acceptance rates
4. **Lattice Energy**: Energy values from lattice averages updated as states are received

In [ ]:
# If you want to stop the simulation early
# runner.stop()

In [ ]:
# Check if the simulation is still running
runner.alive()

In [6]:
# When done, check if it finished properly
runner.finished_gracefully()

All simulation processes finished gracefully


True

## Benefits of Immediate Logging

The new immediate logging approach provides several advantages:

1. **Real-time monitoring**: See data as soon as it's computed
2. **Lower memory usage**: No need to batch data before logging
3. **Better responsiveness**: TensorBoard updates continuously during simulation
4. **Specific data logging**: Only log what's actually received/computed
5. **Failure resilience**: Data is logged even if simulation crashes partway through